<a href="https://colab.research.google.com/github/harshdeep1211/SELF-DRIVING-CAR/blob/main/driving_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
import imageio
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("/content/drive/MyDrive/data1.txt") as f:
    for line in f:
        xs.append("/content/drive/My Drive/Driving /driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images

num_images = len(xs)
split =0.7

train_xs = xs[:int(len(xs) * split)]
train_ys = ys[:int(len(xs) * split)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)
#numpy.array(Image.fromarray(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]).resize([66, 200])).

#  x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(np.array(Image.fromarray(imageio.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]).resize([66, 200])) /255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(np.array(Image.fromarray(imageio.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:]).resize([66, 200])) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

In [ ]:
xs, ys = LoadTrainBatch(2)


<ipython-input-1-d2ae4096f210>:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x_out.append(np.array(Image.fromarray(imageio.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]).resize([66, 200])) /255.0)


In [ ]:
import tensorflow as tf
import scipy
tf.compat.v1.disable_eager_execution()

def weight_variable(shape):
  initial = tf.random.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x1 = tf.compat.v1.placeholder(tf.float32, shape=[None, 200,66, 3])
y1_ = tf.compat.v1.placeholder(tf.float32, shape=[None, 1])

x_image = x1

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = 0.5                                # dropout of 50%
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.keras.activations.linear(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2)

In [ ]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
#import model
import cv2
from subprocess import call
import math

sess=tf.compat.v1.InteractiveSession()


model = tf.train.load_checkpoint(
    "/content/drive/My Drive /Driving /model.ckpt"
)
img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape
keep_prob =  tf.compat.v1.placeholder(tf.float32)

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("/content/drive/MyDrive/data1.txt") as f:
    for line in f:
        xs.append("/content/drive/My Drive/Driving /driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    # np.array(Image.fromarray(imageio.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]).resize([66, 200])) /255.0)

    full_image = imageio.imread("/content/drive/My Drive/Driving /driving_dataset/" + str(i) + ".jpg", pilmode="RGB")
    image =  np.array(Image.fromarray(full_image[-150:]).resize([66, 200])) / 255.0
    degrees = y.eval(feed_dict={x1: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

In [ ]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import scipy.misc
import tensorflow as tf
import scipy
import random
import numpy as np
from __future__ import division
import datetime

import os
import numpy as np
import random

from scipy import pi
from itertools import islice
import cv2
from subprocess import call
from __future__ import division

import os
import numpy as np
import random

from scipy import pi
from itertools import islice
startTime = datetime.datetime.now()
print("Current Time = ",startTime)

LOGDIR = '/content/drive/My Drive/log'

sess = tf.compat.v1.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.compat.v1.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y1_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.compat.v1.train.AdamOptimizer(0.0001).minimize(loss)
sess.run(tf.compat.v1.initialize_all_variables())

# create a summary to monitor cost tensor
tf.compat.v1.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.compat.v1.summary.merge_all()

saver = tf.compat.v1.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs_assignment'
summary_writer = tf.compat.v1.summary.FileWriter(logs_path, graph=tf.compat.v1.get_default_graph())

epochs = 7
batch_size = 100
keep_prob =  tf.compat.v1.placeholder(tf.float32)
# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    print("TRAINING FOR SELF DRIVING CAR BATCH : " , i)
    print(np.shape(x1) , np.shape(y1_))
    train_step.run(feed_dict={x1: xs, y1_: ys, keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={x1:xs, y1_: ys, keep_prob: 1.0})
      print("------> Epoch: %d\n, ------> Step: %d\n, ------> Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x1:xs, y1_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)


print("Time taken for creation of dataframe is {}".format(datetime.datetime.now() - startTime))

Current Time =  2023-07-15 08:53:42.111190


/usr/local/lib/python3.10/dist-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Instructions for updating:
Use `tf.global_variables_initializer` instead.
<ipython-input-1-d2ae4096f210>:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x_out.append(np.array(Image.fromarray(imageio.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:]).resize([66, 200])) /255.0)


TRAINING FOR SELF DRIVING CAR BATCH :  0
(None, 200, 66, 3) (None, 1)


<ipython-input-1-d2ae4096f210>:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x_out.append(np.array(Image.fromarray(imageio.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:]).resize([66, 200])) / 255.0)


------> Epoch: 0
, ------> Step: 0
, ------> Loss: 25.4898


TRAINING FOR SELF DRIVING CAR BATCH :  1
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  2
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  3
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  4
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  5
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  6
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  7
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  8
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  9
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  10
(None, 200, 66, 3) (None, 1)
------> Epoch: 0
, ------> Step: 10
, ------> Loss: 11.0037
TRAINING FOR SELF DRIVING CAR BATCH :  11
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  12
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING CAR BATCH :  13
(None, 200, 66, 3) (None, 1)
TRAINING FOR SELF DRIVING 

In [ ]:
!pip install kora


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00



In [ ]:
!pip install google-colab-shell
from google_colab_shell import getshell

getshell()
getshell(height=400)

In [ ]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)